In [ ]:
from selenium import webdriver
import base64, pickle, time, os
from IPython.display import Image
from IPython.display import display
from PIL import Image as pil
from selenium.webdriver.chrome.options import Options

### 1. Get Youtube Images

##### (1) Get Youtube Driver & Save Element Screen

In [ ]:
def getDriver(youtube_url):
    chrome_options = Options()
    chrome_options.add_argument("--window-size=1920,1080")
    driver = webdriver.Chrome(chrome_options=chrome_options)
#    driver =  webdriver.Chrome()
    driver.get(youtube_url)
    return driver

def saveScreen(driver, filename):
    element = driver.find_element_by_css_selector('#movie_player')
    location = element.location
    size = element.size
    driver.save_screenshot('screenshot.png')
    im = pil.open('screenshot.png')
    
    left = location['x']
    top = location['y']
    right = left + size['width']
    bottom = top + size['height']

    # left, top, right, bottom
    im = im.crop((left, top, right, bottom)) 
    im.save(filename) 

##### (2) Check Element

In [ ]:
def checkElement(driver, selector):
    result = True
    try:
        driver.find_element_by_css_selector(selector)
    except:
        result = False
    return result

##### (3) Skip Ad

In [ ]:
skipAd_selector = ".videoAdUiSkipButton.videoAdUiAction.videoAdUiFixedPaddingSkipButton"
closeAd_selector = ".adDisplay.extra-padding .close-button"

def skipAd(driver):
    if checkElement(driver, skipAd_selector):
        driver.find_element_by_css_selector(skipAd_selector).click()
    
def closeAd(driver):
    if checkElement(driver, closeAd_selector):
        driver.find_element_by_css_selector(closeAd_selector).click()

##### (4) Make Directory

In [ ]:
def makeDir(youtube_url):
    dirList = os.listdir("data/images/")
    dirName = youtube_url.split("v=")[1]
    isDir = True
    if dirName not in dirList:
        os.makedirs("data/images/" + dirName)
        isDir = False
    return isDir, dirName

##### (5) screenshot youtube

In [ ]:
def saveImages(youtube_url, start_term, image_count, save_term, dirName):
    
    # open driver
    driver = getDriver(youtube_url)

    # skip ad
    time.sleep(6)
    skipAd(driver)
    
    time.sleep(6)
    closeAd(driver)

    # save images
    time.sleep(start_term)
    for idx in range(1, image_count+1):
        saveScreen(driver, 'data/images/' + dirName + '/screenshot' + str(idx) + '.png')
        time.sleep(save_term)
        print('saved : screenshot' + str(idx) + '.png')
    # close driver
    driver.close()

##### (6) resizing image (512, 512)

In [ ]:
def makeTunmnail(image_count, dirName):
# make image list
    filenames = []
    for idx in range(1, image_count + 1):
        filename = 'C:/Users/hjnam/study/crawling/data/images/' + dirName  + '/screenshot' + str(idx) + '.png'
        filenames.append(filename)

    # make thumnail
    size = (512, 512)
    thumnails = []
    for filename in filenames:
        im = pil.open(filename)
        im.thumbnail(size)
        savename = filename.split("/")
        savename[len(savename)-1] = savename[len(savename)-1].replace(".png","_thumbnail.png")
        savename = "/".join(savename)
        im.save(savename, "png")
        thumnails.append(savename)
        
    return thumnails

In [ ]:
#youtube_url = "https://www.youtube.com/watch?v=xyq2T_K6DHg" # 이미지 생성 youtube URL 

# 영국남자
#youtube_url = "https://www.youtube.com/watch?v=wUfqAuUPFYE&start=30"

# 태양의후예
youtube_url = "https://www.youtube.com/watch?v=RVVAclDe9cU&start=50"

# 도깨비 
#youtube_url = "https://www.youtube.com/watch?v=2kZg7byEYGA"

### 2. Google Vision API

In [ ]:
# single thread
def vision(filenames):
    url = 'https://cloud.google.com/vision/'

    # open site
    driver =  webdriver.Chrome()
    driver.get(url)

    # enter iframe
    iframe = driver.find_element_by_css_selector("#vision_demo_section iframe")
    driver.switch_to_frame(iframe)
    
    for filename in filenames:
    
        # file upload
        driver.find_element_by_id("input").send_keys(filename)
        resultData = {}

        # check done analytics
        delay = 0
        for _ in range(60):
            if driver.find_element_by_css_selector("#results").text != '':
                break
            time.sleep(1)
            delay += 1
        print("Analytics Delay Time : {} sec".format(delay))
        
        if delay >= 60:
            continue
        
        # tab change (to Safe Search)
        driver.find_element_by_css_selector('[data-type="safeSearchAnnotation"]').click()
        rows = driver.find_elements_by_css_selector('.row.style-scope.vs-safe')
        
        for row in rows:
            for idx in range(len(rows)):
                name = driver.find_elements_by_css_selector('.row.style-scope.vs-safe #label')[idx].text
                state = driver.find_elements_by_css_selector('.row.style-scope.vs-safe #text')[idx].text
                if "Likely" in state or "Possible" in state :
                    state = 2
                elif "Very Unlikely" in state:
                    state = 0
                elif "Unlikely" in state:
                    state = 1
                else:
                    state = 10
                resultData[name] = state
      
        print(resultData) 
        # close image
        driver.find_element_by_css_selector("#exit").click()
    driver.close()
    return resultData

In [ ]:
%%time
isDir, dirName = makeDir(youtube_url)
start_term = 10 # 20초 후 부터 이미지 생성
image_count = 20 # 10개의 이미지 생성
save_term = 0.2 # 저장 텀
saveImages(youtube_url, start_term, image_count, save_term, dirName)

# make thumnail
thumnails = makeTunmnail(image_count, dirName)

In [ ]:
%%time
resultData = vision(thumnails)
#result = vision2(thumnails)